In [0]:
import dataiku
from langchain.chains.question_answering import load_qa_chain
from dataiku.langchain.dku_llm import DKUChatLLM
import json
KB_IDs = {
    "tech_docs": "zQ92IhQ9",
    "non_conformities": "WnKb6p17"
}

client = dataiku.api_client()
project = client.get_default_project()

In [0]:
# Listing available LLMs
llm_list = project.list_llms()

for llm in llm_list:
    print(f"- {llm.description} (id: {llm.id})")

In [0]:
# Fill with your LLM id
LLM_ID = "openai:OpenAI-FA:gpt-4o-mini"

In [0]:
# Preparing the Knowledge Bank, Vector store and LLM
KBs = {
    key: dataiku.KnowledgeBank(id=value, project_key=project.project_key)
    for key, value in KB_IDs.items()
}
vector_stores = {
    key: value.as_langchain_vectorstore()
    for key, value in KBs.items()
}

gpt_lc = DKUChatLLM(llm_id=LLM_ID, temperature=0)

In [0]:
# Create the question answering chain
chain = load_qa_chain(gpt_lc, chain_type="stuff")
query = "Fuel Voltage Levels Quality"


In [0]:
search_results = [result for key, value in vector_stores.items() for result in value.similarity_search(query)]
search_results = [ {
        "doc": s.metadata['doc'],
        "chunk_id": s.metadata['chunk_id'],
        "chunk": s.page_content
    }
    for s in search_results
]
print(search_results)

search_results = {
    key: value.similarity_search(query)
    for key, value in vector_stores.items()
}

for key in KB_IDs:
    for search_result in search_results[key]:
        print(f"# {search_result.doc} \n{search_result.page_content}\n")

In [0]:
print(dict(search_result))

In [0]:
# ⚡ Get the results ⚡
resp = chain({"input_documents":search_results, "question": query})
print(resp["output_text"])